In [0]:
#https://www.kaggle.com/stkbailey/teaching-notebook-for-total-imaging-newbies

In [4]:
!pip install tqdm
!pip install pathlib
!pip install imageio
!pip install kaggle
!mkdir -p /content/.kaggle
!wget https://www.dropbox.com/s/xbbyul14i3zrjd5/kaggle.json
!mv kaggle.json /content/.kaggle/
!chmod 600 /content/.kaggle/kaggle.json
!rm -rf /content/.kaggle/competitions/data-science-bowl-2018/*
!kaggle competitions download -c data-science-bowl-2018

    100% |████████████████████████████████| 3.3MB 334kB/s 
  Running setup.py bdist_wheel for imageio ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/2a/87/cf/7c098611c6b49ceb8967ff9c22277a8d3ba6d9d6742068a33d
Successfully built imageio
--2018-03-12 11:18:38--  https://www.dropbox.com/s/xbbyul14i3zrjd5/kaggle.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/lCSjQG5QPqJdyJOwhbZHJz3gcCrKLKtH7Y0OCmRkkd9foI1HKoVxqTy2HhEm9fOC/file [following]
--2018-03-12 11:18:38--  https://dl.dropboxusercontent.com/content_link/lCSjQG5QPqJdyJOwhbZHJz3gcCrKLKtH7Y0OCmRkkd9foI1HKoVxqTy2HhEm9fOC/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.6.6, 2620:100:601c:6::a27d:606
Connecting to dl.dropboxusercontent.com (dl.dropboxusercon

In [0]:
!unzip /content/.kaggle/competitions/data-science-bowl-2018/stage1_sample_submission.csv.zip -d /content/.kaggle/competitions/data-science-bowl-2018
!unzip /content/.kaggle/competitions/data-science-bowl-2018/stage1_train_labels.csv.zip -d /content/.kaggle/competitions/data-science-bowl-2018
!unzip /content/.kaggle/competitions/data-science-bowl-2018/stage1_test.zip -d /content/.kaggle/competitions/data-science-bowl-2018/stage1_test
!unzip /content/.kaggle/competitions/data-science-bowl-2018/stage1_train.zip -d /content/.kaggle/competitions/data-science-bowl-2018/stage1_train

In [0]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '../content/.kaggle/competitions/data-science-bowl-2018/stage1_train/'
TEST_PATH = '../content/.kaggle/competitions/data-science-bowl-2018/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [12]:
import pathlib
import imageio
import numpy as np

# Glob the training data and load a single image path
training_paths = pathlib.Path('../content/.kaggle/competitions/data-science-bowl-2018/stage1_train').glob('*/images/*.png')
# training_paths
training_sorted = sorted([x for x in training_paths])
training_sorted
# im_path = training_sorted[0]
# # im = imageio.imread(str(im_path))

[]